# Series Resistor Implementation
<a href="http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=4119950">Dylan Williams' Original Paper</a>

### Other References
1. How Radical Solves the Problem
2. A python version

### Some Notes About Radical Implementation
The function returns a modified structure ref. The structure is first passes as a variable and then the solution and analysis is appended then the structure is returned with these new attributes. This opens an interesting question, is a calibration a stateless function or a class? The RadicalData.mat is a serialization of a class, but SRAlgorithmV3 is a stated function (really a method ) on the Ref object. 

In this case, I think the series resistor experiment (analysis) is a class and the implemented Series Resistor is a class method requiring the attributes, self.series_model, self.short_model or self.reflect_model, self.thru_measurement, self.short_measurement, self.series_resistor_measurement. This method should be self.calculate_series_resistor_correction() 

It is usefull that 
```python
def calculate_series_resistor_correction(thru_measurement,short_measurement,series_measurement,series_model,short_model=None):
```
is a stateless function that returns an eight term correction in [[frequency,S1_11,S1_21,S1_12,S1_22,S2_11,S2_21,S2_12,S2_22]..]
form. 

This brings up the need for a TwoPortSparameterModel, or a class that can be passed a frequency list and return a S-parameter list. 
The TwoPortModel should have the following attributes:
1. self.frequency a frequency list. the alias could be self.f, it should be setable like DataSimulator
2. self.port_1_impedance and self.port_2_impedance
3. self.function_list, a list of functions, FunctionalModels or DataSimulator objects that create a SParameter.
4. The ability to represent itself in any form using TwoPortGraph, this brings up the need to set the port 1 and port 2 impedances in the graph. Not currently supported. 
5. The ability to output a latex form of the matrix in equations
6. A serialized form that can be saved and opened -> this is a little harder.
7. A string form that is [[f1,S11,S12,S21,S22]...[fN,S11,S12,S21,S22]] that can be passed to the SNP class for text form manipulations
8. self.frequency_units, and the ability to change the frequency units. It should fix the equations to the frequency units

Some other thoughts about functions. Functions that operate on objects should not change the state of the object, and should return a new object. Scripts and methods can change the state of the object.

Also this brings up the fundemental data model of a sparameter. Is it the sparameter_complex list? since this is immutable under all but frequency unit changes, it is also the data that is used in correct_sparameters.  

## Radical Solution in SRAlgorithmV3.mat

```octave

function Ref = SRAlgorithmV3(Ref)%input are the T-Matrices for the different Ref Standards
%SYNTAX:     
%   SRAlgorithmV3(Ref)
%   This algorithm obtains the calibration coefficients with the series
%   resistor calibration algorithm, which requires a thru, reflect and
%   series resistor. This code also allows a user to model the short as an
%   resistor with an inductor in series.
%
% If you find any errors, please let me know: nathan dot orloff at gmail dot com
% 
% INPUT
% Ref:
%   Models
%   Ref.RsModel = Model for Series Resistor. S-Parameter
%   Ref.S1Model (optional) = Model for Short Circuit Reflect. S-Parameter
%   Data
%   Ref.Tt = Measurement for Thru. T-Parameter
%   Ref.Ts = Measurement for Short Circuit Reflect. T-Parameter
%   Ref.Trs = Measurement for Series Resistor. T-Parameter
%
% OUTPUT
% Ref:
%   Outputs the calibration from RadiCal.
%   Ref.S1 = Port 1 error box. S-Parameter
%   Ref.S2 = Port 2 error box. S-Parameter
%   Ref.T1 = Port 1 error box. T-Parameter
%   Ref.T2 = Port 2 error box. T-Parameter

% Copyright (C) Nathan D. Orloff, 2011.

% HISTORY:
% Version 1.0, 2011-05-21.
%[1]-SEE BELOW
Trsdc = s2t(Ref.RsModel);
Ttdc = s2t(repmat([0 1 1 0],size(Ref.Tt,1),1));
if ~isfield(Ref,'S1Model')
    Ssdc = repmat([-1 0 0 -1],size(Ref.Tt,1),1);
else
    Ssdc = Ref.S1Model;
end
Ss = Ref.Ss;
Tti = invT(Ref.Tt);%inverse of the Thru T Matrix
M = prodT(Ref.Trs,Tti,Ttdc);%product of the Tsr*Tt^-1
for i = 1:size(Ref.Tt,1)%Num of Freqs
    %Trs*Tti*Ttdc*X=X*Trsdc [eq 1]
    %This a the LHS of the series of algebraic equations which
    %characterize the error boxes
    Q = [-1*Trsdc(i,3),M(i,2),0;...                 % Constraints placed by the series resistor; eq. 6 in DOI:10.1109/ARFTG.1997.327267
        0,M(i,4)-Trsdc(i,1),-1*Trsdc(i,3);...       % eq. 7
        M(i,1)-Trsdc(i,4),0,M(i,2);...              % eq. 8
        M(i,3),-1*Trsdc(i,2),M(i,4)-Trsdc(i,4);...  % eq. 9
        1,-1*Ssdc(i,1)*Ss(i,1),-1*Ss(i,1);...       % eq. 10, Constraints placed by the short
        Ssdc(i,4)*(Ss(i,4)*Tti(i,1)-Tti(i,3)),Ss(i,4)*Tti(i,2)-Tti(i,4),Ssdc(i,4)*(Ss(i,4)*Tti(i,2)-Tti(i,4))]; % eq. 11
    %RHS
    T = [Trsdc(i,1)-M(i,1);-1*M(i,3);Trsdc(i,2);0;...
        -1*Ssdc(i,1);Tti(i,3)-Ss(i,4)*Tti(i,1)];%RHS
    A = Q\T;%Solution vector *least squares algorithm
    Ref.Error(i,:) = T-Q*A;%Error in the method
    T1(i,:) = (1./(sqrt(A(3)-A(1).*A(2))))*[1,A(1),A(2),A(3)];%Error Box 1 assignment in T-matrix Form!
end
clear Q A T M

%[2]-We can use S1 to obtain S2
Ref.S1 = t2s(T1);%convert to S
Ref.T1 = T1;
Ref.T2 = prodT(invT(Ref.T1),invT(Tti),Ttdc);%see page 133 in Orloff 2 for derivation
Ref.S2 = t2s(Ref.T2);%convert to S
Ref = rmfield(Ref,'Trs');
Ref = rmfield(Ref,'Tt');
Ref = rmfield(Ref,'Ss');

%[1]-For the series resistor calibration we need the following T-Matrices:
%Series Resistor(measured), Thru(measured), Short aka Reflect(measured),
%Perfect Short(generated). Below is a description of the symbols used;
%Ref.Tsr = Series Resistor(measured);
%Ref.Tt=Thru(measured);
%Ref.Ts=Short aka Reflect(measured);
%Ref.Tsrdc = Series Resistor(modeled)- DC behavior graphed to TRL HF;
%Ref.Ttdc = Thru(modeled);
%Ref.Tsdc = Short(modeled);
%[2]-Understand that S2 which we save here is different that the actual error
%box. What we save is such that 
%Mcalibrated = t2s(prodT(invT(s2t(S1)),s2t(Mmeasured),invT(s2t(S2))));
```

In [ ]:
class TwoPortSparameterModel():
    """TwoPortSparameterModel is a class for generating modelled s parameters.
    
    """
    def __init__(self,file_path=None,**options):
        """Intializes the TwoPortSparameter Class"""
        defaults={"equation_list":[]}
        